In [18]:
import neuroglancer
import cloudvolume
import pandas as pd, os

## README
This example assumes you have precomputed datasets already made and hosted with cloudvolume.
To host the raw data in this example that I already converted to precomputed format, open an ipython terminal and do:
```python
import cloudvolume
vol = cloudvolume.CloudVolume('file:///home/wanglab/Documents/neuroglancer/an19/647')
vol.viewer(port=1337) # You can choose the port here, but remember it for later
```
The last command will cause the window to hang -- that is the expected behavior. Do not try to do this in this jupyter notebook as it will cause your session to hang and you won't be able to run any of the following steps.

## Set the neuroglancer client you want to use. 
Default is Seung lab's which should work fine for this example

In [19]:
neuroglancer.set_static_content_source(url='https://neuromancer-seung-import.appspot.com')

# Load in the raw data into neuroglancer and generate the link

In [20]:
# This volume handle can be used to notify the viewer that the data has changed.
viewer = neuroglancer.Viewer()

# Load in a segmentation layer (e.g. the raw-space allen atlas) that is being hosted with cloudvolume
with viewer.txn() as s:
    s.layers['an19'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1337',
    shader = '''
    void main() {
  float v = toNormalized(getDataValue(0)) * 20.0;
  emitRGBA(vec4(v, v, v, v));
}
'''                                                 
    )
print(viewer) # Click the link that is generated to see your volume displayed in Neuroglancer

http://127.0.0.1:39341/v/1ffee83305e17628ba3a9719ac06d3947ca84892/


## Host the raw-space allen atlas cloudvolume

You will need to host this with cloudvolume as was done for the raw data above. Use a different port though, e.g.
```python
import cloudvolume
vol = cloudvolume.CloudVolume('file:///home/wanglab/Documents/neuroglancer/an19/atlas')
vol.viewer(port=1338) # must be a different port than the raw data
```
Again, make sure to do this outside of this jupyter notebook

## Load in the atlas
Load in a segmentation layer (e.g. the raw-space allen atlas) that is being hosted with cloudvolume using the same viewer object that you already made. If you go back to the neuroglancer window after running this it should be there in a new layer

In [22]:
with viewer.txn() as s:
    s.layers['rawatlas'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338'
    )
    

Got my-action 1
  Layer selected values: Map({"an19": 1250, "rawatlas": 334})
Got my-action 2
  Layer selected values: Map({"an19": 1328, "rawatlas": 131})
Got my-action 3
  Layer selected values: Map({"an19": 1331, "rawatlas": 451})
Got my-action 4
  Layer selected values: Map({"an19": 1232, "rawatlas": 334})
Got my-action 5
  Layer selected values: Map({"an19": 1342, "rawatlas": 311})
Got my-action 6
  Layer selected values: Map({"an19": 1299, "rawatlas": 780})
Got my-action 7
  Layer selected values: Map({"an19": 1694, "rawatlas": 663})
Got my-action 8
  Layer selected values: Map({"an19": 1661, "rawatlas": 655})
Got my-action 9
  Layer selected values: Map({"an19": 1816, "rawatlas": 788})
Got my-action 10
  Layer selected values: Map({"an19": 2238, "rawatlas": 961})
Got my-action 11
  Layer selected values: Map({"an19": 1301, "rawatlas": 303})
Got my-action 12
  Layer selected values: Map({"an19": 1519, "rawatlas": 403})
Got my-action 13
  Layer selected values: Map({"an19": 1236, 

Got my-action 105
  Layer selected values: Map({"an19": 3417, "rawatlas": 10703})
Got my-action 106
  Layer selected values: Map({"an19": 2201, "rawatlas": 423})
Got my-action 107
  Layer selected values: Map({"an19": 1348, "rawatlas": 672})
Got my-action 108
  Layer selected values: Map({"an19": 1147, "rawatlas": 149})
Got my-action 109
  Layer selected values: Map({"an19": 1374, "rawatlas": 1022})
Got my-action 110
  Layer selected values: Map({"an19": 1432, "rawatlas": 1022})
Got my-action 111
  Layer selected values: Map({"an19": 1341, "rawatlas": 574})
Got my-action 112
  Layer selected values: Map({"an19": 2341, "rawatlas": 771})
Got my-action 113
  Layer selected values: Map({"an19": 1129, "rawatlas": 930})
Got my-action 114
  Layer selected values: Map({"an19": 1205, "rawatlas": 67})
Got my-action 115
  Layer selected values: Map({"an19": 1080, "rawatlas": 128})
Got my-action 116
  Layer selected values: Map({"an19": 1095, "rawatlas": 930})
Got my-action 117
  Layer selected va

Use the look-up table to make a key binding to the Neuroglancer session.

In [21]:
# First you need to create a dictionary mapping the atlas id to the region name. 
# Here's my example

csv_file = '/jukebox/LightSheetTransfer/atlas/ls_id_table_w_voxelcounts_16bit.xlsx'

df = pd.read_excel(csv_file)

atlas_df = df.copy()

atlas_dict = {}
for ind, row in atlas_df.iterrows():
    atlas_dict[int(row['id'])] = row['name']
    
# Here is the actual code for making the key binding. Copy and paste this
# into your jupyter notebook where you have already made the viewer() object

num_actions = 0
def my_action3(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
        region_id = s.selected_values['rawatlas']
        region_name = atlas_dict.get(region_id)
        st.status_messages['hello'] = ('%i:%s' %
                                    (region_id, region_name))

    print('Got my-action %i' % num_actions)
#     print('  Mouse position: %s' % (s.mouse_voxel_coordinates,))
    print('  Layer selected values: %s' % (s.selected_values,))
viewer.actions.add('my-action3', my_action3)

with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyp'] = 'my-action3'
    s.status_messages['hello'] = 'Welcome to the segment labeling example. Press "p" to see region name under cursor.'

Adjust the viewer and take a screenshot. Good for videos!

In [6]:
svdst = '/home/wanglab/Documents/neuroglancer/screenshots/20161205_tp_bl6_lob45_1000r_01/amygdala'
if not os.path.exists(svdst): os.mkdir(svdst)
for i in range(443,763):
    with viewer.txn() as s:
        s.voxel_coordinates = [2983,3429,i]
    ss = neuroglancer.ScreenshotSaver(viewer, svdst)
    ss.capture(index=i)   


Change the view layout to show the segmentation side by side with the image, rather than overlayed.  This can also be done from the UI by dragging and dropping.  The side by side views by default have synchronized position, orientation, and zoom level, but this can be changed.

In [ ]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['20161205_tp_bl6_lob45_1000r_01', 'overlay']),
         neuroglancer.LayerGroupViewer(layers=['rawatlas'])])

Remove the overlay layer.

In [ ]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['20161205_tp_bl6_lob45_1000r_01']),
         neuroglancer.LayerGroupViewer(layers=['rawatlas'])])